In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pyodbc
from sklearn.preprocessing import MinMaxScaler , RobustScaler , StandardScaler
from sklearn import metrics


In [2]:
server = "192.168.10.35"
database = "LiveTseDB"
username = "mostafa"
password = "mostafa1234"
driver = "ODBC Driver 18 for SQL Server"

conn_str = (
    f"DRIVER={{{driver}}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"Encrypt=yes;"
    f"TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)

query = """SELECT [ID]
      ,[Namad]
      ,[StockID]
      ,[NotiTypeName]
      ,[NotiTypeID]
      ,[Time]
      ,[Date]
      ,[LastUpdate]
      ,[IsEvaluate]
      ,[MaxDay0]
      ,[MaxDay1]
      ,[MaxDay2]
      ,[MaxDay3]
      ,[MaxDayPercent0]
      ,[MaxDayPercent1]
      ,[MaxDayPercent2]
      ,[MaxDayPercent3]
      ,[OnEventPrice]
      ,[OnEventPricePercent]
      ,[YeserdayPrice]
      ,[MaxGain]
      ,[MaxGainPosition]
  FROM [LiveTseDB].[dbo].[Notifications] where IsEvaluate = 1;"""

df = pd.read_sql_query(query, conn)
data = pd.read_sql_query(query, conn)

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_13000\1449392340.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
C:\Users\Mesbah\AppData\Local\Temp\ipykernel_13000\1449392340.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


In [3]:
df.drop(
    [
        "LastUpdate",
        "MaxDay0",
        "MaxDay1",
        "MaxDay2",
        "MaxDay3",
        "ID",
        "IsEvaluate",
        "Namad",
        "NotiTypeName",
        "MaxGainPosition",
        "OnEventPrice",
        "OnEventPricePercent",
        "Time",
        "MaxDayPercent0",
        "MaxDayPercent1",
        "MaxDayPercent2",
        "MaxDayPercent3",
    ],
    inplace=True,
    axis=1,
)

In [4]:
df["Date"] = pd.to_datetime(df["Date"])
df['Date'] = df['Date'].astype('int64') // 10**9 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55781 entries, 0 to 55780
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StockID        55781 non-null  int64  
 1   NotiTypeID     55781 non-null  object 
 2   Date           55781 non-null  int64  
 3   YeserdayPrice  55781 non-null  int64  
 4   MaxGain        55781 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 2.1+ MB


In [6]:
df.head(11)

,StockID,NotiTypeID,Date,YeserdayPrice,MaxGain
0,17914401175772326,010E7B33-4D0A-483C-B7A2-53BD6F19A0EC,1759795200,23130,5.732218
1,21426277483799140,666D8D60-0F30-46AB-BA13-FF10F271B90C,1759536000,3809,7.468774
2,59522975635235187,666D8D60-0F30-46AB-BA13-FF10F271B90C,1755907200,3482,2.983826
3,42799209630949274,96B64AE4-86DE-4CE2-9612-3E4B80CC7B90,1759622400,10940,7.027027
4,4733285133017464,666D8D60-0F30-46AB-BA13-FF10F271B90C,1759190400,2053,6.224900
5,46752599569017089,96B64AE4-86DE-4CE2-9612-3E4B80CC7B90,1759190400,3305,3.644578
6,17330546482145553,96B64AE4-86DE-4CE2-9612-3E4B80CC7B90,1758931200,1637,10.810811
7,37614886280396031,686075F9-C00F-43B3-8F0D-A99F540B2BAB,1760400000,5020,1.121857
8,44549439964296944,686075F9-C00F-43B3-8F0D-A99F540B2BAB,1759276800,6080,7.827476
9,34718633636164421,6F14AF33-464D-46B9-83A6-8D779D9EFEDF,1757116800,10241,0.009752


In [7]:
# df = pd.get_dummies(df , dtype=int)
# df.info()

In [8]:
# corr = df.corr()
# plt.figure(figsize=(20, 10)) 
# sns.heatmap(corr , annot=True , cmap="Greens" , fmt=".2f")

In [9]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
ohe_data = ohe.fit_transform(df[["NotiTypeID"]])

ohe_df = pd.DataFrame(ohe_data,columns=ohe.get_feature_names_out(["NotiTypeID"]),index=df.index)
    
df = pd.concat([df.drop(columns=["NotiTypeID"]), ohe_df], axis=1)


In [10]:
# df = df.groupby(['StockID', 'Date'], as_index=False).max()

agg_dict = {col: 'min' if col == "MaxGain" else 'max' 
            for col in df.columns if col not in ['StockID','Date']}

# df = df.groupby(['StockID', 'Date']).agg(agg_dict).reset_index()
df = pd.concat([df , df.groupby(['StockID', 'Date']).agg(agg_dict).reset_index()] , ignore_index=True)

In [ ]:
x = df.drop(["MaxGain"], axis=1)
y = df[["MaxGain"]]

In [12]:
x.head()

,StockID,Date,YeserdayPrice,NotiTypeID_010E7B33-4D0A-483C-B7A2-53BD6F19A0EC,NotiTypeID_0AD8302F-D2EC-4E30-A26D-FAB20664ECD8,NotiTypeID_45913250-DA33-4A76-A5C4-D8801423A2C4,NotiTypeID_501BC271-1494-44E1-9ED9-65ECAB0768AE,NotiTypeID_5693C959-C56D-4B9B-889D-CD445130182C,NotiTypeID_666D8D60-0F30-46AB-BA13-FF10F271B90C,NotiTypeID_686075F9-C00F-43B3-8F0D-A99F540B2BAB,NotiTypeID_6F14AF33-464D-46B9-83A6-8D779D9EFEDF,NotiTypeID_78F18466-0917-4D2A-AB9F-E2832ADF92A3,NotiTypeID_889313DF-C9E5-45C4-862F-80092885A320,NotiTypeID_96B64AE4-86DE-4CE2-9612-3E4B80CC7B90,NotiTypeID_A68DBD50-3858-476F-B023-67360C2A4BF1,NotiTypeID_CC3EAC84-AA63-4318-8035-AED42A7FF0F9,NotiTypeID_DDC22661-5E87-402D-A487-2CBC46532686,NotiTypeID_EFD74134-5A87-46AC-8A32-9A541497748A
0,17914401175772326,1759795200,23130,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,21426277483799140,1759536000,3809,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,59522975635235187,1755907200,3482,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,42799209630949274,1759622400,10940,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,4733285133017464,1759190400,2053,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [13]:
y.shape

(68430, 1)

In [14]:
x.shape

(68430, 18)

In [15]:
# mm = MinMaxScaler()

# x = mm.fit_transform(x)

# rs = RobustScaler()

# x = rs.fit_transform(x)

ss = StandardScaler()

x = ss.fit_transform(x)

In [16]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=10)
# x = pca.fit_transform(x)

In [17]:
# explained_var = np.cumsum(pca.explained_variance_ratio_)

# plt.plot(range(1, len(explained_var)+1), explained_var, marker='o')
# plt.xlabel('تعداد مؤلفه‌ها')
# plt.ylabel('واریانس تجمعی توضیح داده‌شده')
# plt.title('نمودار واریانس تجمعی (Cumulative Explained Variance)')
# plt.grid(True)
# plt.show()

In [18]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(x , label = y)

data_dmatrix

In [20]:
X_train , X_test , Y_train , Y_test = train_test_split(x,y , test_size=0.1 , random_state=42)


In [21]:
X_train.shape

(47326, 18)

In [22]:
# reg_mod = xgb.XGBRegressor(
#     n_estimators=600,
#     learning_rate=0.1,
#     subsample=0.8,
#     colsample_bytree=1,
#     max_depth=5,
#     gamma=0,
# )
# reg_mod.fit(X_train, Y_train)

In [23]:
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'max_depth': [3,4,5],
#     'learning_rate': [0.01,0.05,0.1],
#     'n_estimators': [100, 300, 500],
#     'subsample': [0.7,0.8,1.0],
#     'colsample_bytree': [0.7,0.8,1.0]
# }

# grid = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='r2')
# grid.fit(X_train, Y_train)
# print(grid.best_params_)

In [118]:
# reg_mod.fit(X_train , Y_train)

# predictions = reg_mod.predict(X_test)

In [119]:
# print("R2 : " , metrics.r2_score(Y_test , predictions))
# print("MAE : " , metrics.mean_absolute_error(Y_test , predictions))
# print("MSE : " , metrics.mean_squared_error(Y_test , predictions))
# print("RMSE : " , np.sqrt(metrics.mean_squared_error(Y_test , predictions)))

In [120]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(
#     400,
#     random_state=0,
# )

# rf.fit(X_train, Y_train)

In [121]:
# rf_predict = rf.predict(X_test)

In [122]:
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(rf, X_test, Y_test,cv=10)
# print("Mean cross-validation score: %.2f" % scores.mean())

In [123]:
# print("R2 : " , metrics.r2_score(Y_test , rf_predict))
# print("MAE : " , metrics.mean_absolute_error(Y_test , rf_predict))
# print("MSE : " , metrics.mean_squared_error(Y_test , rf_predict))
# print("RMSE : " , np.sqrt(metrics.mean_squared_error(Y_test , rf_predict)))

In [124]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

rf = RandomForestRegressor(
    n_estimators=750,
    random_state=42,
    max_features="sqrt",
    min_samples_split=2,
    min_samples_leaf=1,
)

xgb_model = xgb.XGBRegressor(
    n_estimators=750,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=1,
    reg_alpha=0,
    reg_lambda=1,
    random_state=42,
    max_depth=6,
)

ridge = Ridge(alpha=10)

mlp = MLPRegressor(
    hidden_layer_sizes=(64, 64, 32),
    activation="tanh",
    solver="adam",
    learning_rate_init=0.005,
    max_iter=500,
    random_state=42,
)

svr = SVR(kernel='rbf', C=100, gamma=1, epsilon=0.2)

# --- Stacking Regressor ---
stacked_model = StackingRegressor(
    estimators=[("rf", rf), 
                ("xgb", xgb_model), 
                ("ridge", ridge),
                ("mlp", mlp),
                ("svr", svr),
                ],
    final_estimator=Ridge(),
    cv=7,
    n_jobs=-1,
)

# multi_stacked = MultiOutputRegressor(stacked_model)

In [ ]:
# stacked_model.fit(X_train, Y_train)

c:\Users\Mesbah\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\ensemble\_stacking.py:1041: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,estimators,"[('rf', ...), ('xgb', ...), ...]"
,final_estimator,Ridge()
,cv,7
,n_jobs,-1
,passthrough,False
,verbose,0
,n_estimators,750
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1


In [126]:
# cv_scores = cross_val_score(stacked_model, X_train, Y_train, cv=10, scoring='r2')
# print(f"Mean CV R2: {np.mean(cv_scores):.4f}")

In [127]:
y_train_pred = stacked_model.predict(X_train)
y_test_pred = stacked_model.predict(X_test)

In [128]:
print("train score:")
print("R2 : " , metrics.r2_score(Y_train , y_train_pred))
print("MAE : " , metrics.mean_absolute_error(Y_train , y_train_pred))
print("MSE : " , metrics.mean_squared_error(Y_train , y_train_pred))
print("RMSE : " , np.sqrt(metrics.mean_squared_error(Y_train , y_train_pred)))
print("test score:")
print("R2 : " , metrics.r2_score(Y_test , y_test_pred))
print("MAE : " , metrics.mean_absolute_error(Y_test , y_test_pred))
print("MSE : " , metrics.mean_squared_error(Y_test , y_test_pred))
print("RMSE : " , np.sqrt(metrics.mean_squared_error(Y_test , y_test_pred)))

train score:
R2 :  0.9243099372254064
MAE :  0.5721829210748557
MSE :  0.6751718760342265
RMSE :  0.8216884300233431
test score:
R2 :  0.8484916877335412
MAE :  0.7891464793294645
MSE :  1.3101138931641751
RMSE :  1.1446020676043596


In [142]:
df.columns

Index(['StockID', 'Date', 'YeserdayPrice', 'MaxGain',
       'NotiTypeID_010E7B33-4D0A-483C-B7A2-53BD6F19A0EC',
       'NotiTypeID_0AD8302F-D2EC-4E30-A26D-FAB20664ECD8',
       'NotiTypeID_45913250-DA33-4A76-A5C4-D8801423A2C4',
       'NotiTypeID_501BC271-1494-44E1-9ED9-65ECAB0768AE',
       'NotiTypeID_5693C959-C56D-4B9B-889D-CD445130182C',
       'NotiTypeID_666D8D60-0F30-46AB-BA13-FF10F271B90C',
       'NotiTypeID_686075F9-C00F-43B3-8F0D-A99F540B2BAB',
       'NotiTypeID_6F14AF33-464D-46B9-83A6-8D779D9EFEDF',
       'NotiTypeID_78F18466-0917-4D2A-AB9F-E2832ADF92A3',
       'NotiTypeID_889313DF-C9E5-45C4-862F-80092885A320',
       'NotiTypeID_96B64AE4-86DE-4CE2-9612-3E4B80CC7B90',
       'NotiTypeID_A68DBD50-3858-476F-B023-67360C2A4BF1',
       'NotiTypeID_CC3EAC84-AA63-4318-8035-AED42A7FF0F9',
       'NotiTypeID_DDC22661-5E87-402D-A487-2CBC46532686',
       'NotiTypeID_EFD74134-5A87-46AC-8A32-9A541497748A'],
      dtype='object')

In [19]:
import joblib
# scaler = joblib.load("scaler.pkl")
model = joblib.load("model.pkl")
# ohe_loaded = joblib.load("ohe_encoder.pkl")

In [ ]:
y_test_pred_2 = model.predict(x)

In [21]:
x

array([[-0.92123564,  0.97861824, -0.14708551, ..., -0.13673957,
        -0.09261748, -0.17284193],
       [-0.75112344,  0.796135  , -0.36351023, ..., -0.13673957,
        -0.09261748, -0.17284193],
       [ 1.09424711, -1.75863029, -0.36717313, ..., -0.13673957,
        -0.09261748, -0.17284193],
       ...,
       [ 1.7007955 ,  0.91779049, -0.31645261, ..., -0.13673957,
        -0.09261748, -0.17284193],
       [ 1.7007955 ,  0.97861824, -0.31634059, ..., -0.13673957,
        -0.09261748, -0.17284193],
       [ 1.7007955 ,  1.34358471, -0.31499641, ...,  7.31317196,
        -0.09261748, -0.17284193]], shape=(68430, 18))

In [22]:
df["StockID"].nunique()

938

In [ ]:
results = []

new_df = df

new_df["y_pred"] = y_test_pred_2


group_cols = [
    "StockID",
    "NotiTypeID_010E7B33-4D0A-483C-B7A2-53BD6F19A0EC",
    "NotiTypeID_0AD8302F-D2EC-4E30-A26D-FAB20664ECD8",
    "NotiTypeID_45913250-DA33-4A76-A5C4-D8801423A2C4",
    "NotiTypeID_501BC271-1494-44E1-9ED9-65ECAB0768AE",
    "NotiTypeID_5693C959-C56D-4B9B-889D-CD445130182C",
    "NotiTypeID_666D8D60-0F30-46AB-BA13-FF10F271B90C",
    "NotiTypeID_686075F9-C00F-43B3-8F0D-A99F540B2BAB",
    "NotiTypeID_6F14AF33-464D-46B9-83A6-8D779D9EFEDF",
    "NotiTypeID_78F18466-0917-4D2A-AB9F-E2832ADF92A3",
    "NotiTypeID_889313DF-C9E5-45C4-862F-80092885A320",
    "NotiTypeID_96B64AE4-86DE-4CE2-9612-3E4B80CC7B90",
    "NotiTypeID_A68DBD50-3858-476F-B023-67360C2A4BF1",
    "NotiTypeID_CC3EAC84-AA63-4318-8035-AED42A7FF0F9",
    "NotiTypeID_DDC22661-5E87-402D-A487-2CBC46532686",
    "NotiTypeID_EFD74134-5A87-46AC-8A32-9A541497748A",
]


def summarize_group(g):
    return pd.Series(
        {
            "MAE": metrics.mean_absolute_error(
                [g["MaxGain"].mean()], [g["y_pred"].mean()]
            ),
            "MaxGain_min": g["MaxGain"].mean(),
            "y_pred_min": g["y_pred"].mean(),
            "Count": len(g),
        }
    )


group_acc = df.groupby(group_cols).apply(summarize_group).reset_index()
group_acc_sorted = group_acc.sort_values(by="MAE", ascending=False).reset_index(
    drop=True
)


# group_acc_sorted[group_acc_sorted["Count"] > 10]["MAE"].mean()

C:\Users\Mesbah\AppData\Local\Temp\ipykernel_13000\315246100.py:60: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_acc = df.groupby(group_cols).apply(summarize_group).reset_index()


In [31]:
group_acc_sorted

,StockID,NotiTypeID_010E7B33-4D0A-483C-B7A2-53BD6F19A0EC,NotiTypeID_0AD8302F-D2EC-4E30-A26D-FAB20664ECD8,NotiTypeID_45913250-DA33-4A76-A5C4-D8801423A2C4,NotiTypeID_501BC271-1494-44E1-9ED9-65ECAB0768AE,NotiTypeID_5693C959-C56D-4B9B-889D-CD445130182C,NotiTypeID_666D8D60-0F30-46AB-BA13-FF10F271B90C,NotiTypeID_686075F9-C00F-43B3-8F0D-A99F540B2BAB,NotiTypeID_6F14AF33-464D-46B9-83A6-8D779D9EFEDF,NotiTypeID_78F18466-0917-4D2A-AB9F-E2832ADF92A3,NotiTypeID_889313DF-C9E5-45C4-862F-80092885A320,NotiTypeID_96B64AE4-86DE-4CE2-9612-3E4B80CC7B90,NotiTypeID_A68DBD50-3858-476F-B023-67360C2A4BF1,NotiTypeID_CC3EAC84-AA63-4318-8035-AED42A7FF0F9,NotiTypeID_DDC22661-5E87-402D-A487-2CBC46532686,NotiTypeID_EFD74134-5A87-46AC-8A32-9A541497748A,MAE,MaxGain_min,y_pred_min,Count
0,37614886280396031,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,31.482325,32.989691,1.507365,1.0
1,37614886280396031,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,31.368394,34.916493,3.548098,1.0
2,7681671915916933,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,11.920314,13.613232,1.692918,1.0
3,67141987086032267,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,11.365302,12.480037,1.114735,1.0
4,67141987086032267,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,10.521061,15.134963,4.613902,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9145,29590002988360984,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.003212,4.107043,4.103830,39.0
9146,37120336886630649,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0.003150,2.781240,2.778090,6.0
9147,52497713622747990,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0.003033,2.206531,2.209564,1.0
9148,30430907997908005,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.002495,2.966102,2.968597,1.0


In [43]:
group_acc_sorted.to_csv("zog.csv" , index=False)

In [129]:
import joblib
# ذخیره مقدار های مهم
# joblib.dump(ss, "scaler.pkl")
# joblib.dump(stacked_model, "model.pkl")
# joblib.dump(ohe, "ohe_encoder.pkl")


# استفاده نشه
# joblib.dump(pca, "pca_model.pkl")

In [130]:
# test_data = (
#     data.drop(
#         [
#             "MaxDayPercent0",
#             "MaxDayPercent1",
#             "MaxDayPercent2",
#             "MaxDayPercent3",
#             "LastUpdate",
#             "Time",
#             "ID",
#             "IsEvaluate",
#             "Namad",
#             "NotiTypeName",
#         ],
#         axis=1,
#     )
#     .iloc[[11]]
# )
# test_data

In [131]:
# test_data = pd.DataFrame(
#     [{
#         "StockID": 25215182208950217,
#         "NotiTypeID": "6F14AF33-464D-46B9-83A6-8D779D9EFEDF",
#         "MaxDay0": 1610,
#         "MaxDay1": 1630,
#         "MaxDay2": 1577,
#         "MaxDay3": 1567,
#         "OnEventPrice": 1630,
#         "OnEventPricePercent": 0.31,
#         "YeserdayPrice": 1610,
#         "MaxGain": 1.4,
#         "MaxGainPosition": 1,
#     }],
# )
# test_data

# load and use model

score :  <br/>
R2 :  0.846434410555688 <br/>
MAE :  0.5316831685418019 <br/>
MSE :  1.304982498541519 <br/>
RMSE :  1.1423583056736266 <br/>

In [132]:
# import joblib
# scaler = joblib.load("scaler.pkl")
# model = joblib.load("model.pkl")
# ohe_loaded = joblib.load("ohe_encoder.pkl")


# استفاده نشه
# # pca_loaded = joblib.load("pca_model.pkl")

In [133]:

# user_input = test_data

# encoded_data = ohe_loaded.transform(user_input[["NotiTypeID"]])

# ohe_df = pd.DataFrame(encoded_data,columns=ohe_loaded.get_feature_names_out(["NotiTypeID"]),index=user_input.index)
    
# result_encoded_data = pd.concat([user_input.drop(columns=["NotiTypeID"]), ohe_df], axis=1)

# user_input_scaled = scaler.transform(result_encoded_data)

# # user_pca = pca_loaded.transform(user_input_scaled)


In [134]:
# prediction = model.predict(user_input_scaled)
# print("result:", prediction)

In [135]:
# test_res_data = df.iloc[[11]][["MaxDayPercent0" , "MaxDayPercent1" , "MaxDayPercent2" , "MaxDayPercent3"]].values
# test_res_data

In [136]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(random_state=42)

# rf_param_grid = {
#     "n_estimators": [200, 500, 800],
#     "max_depth": [None, 10, 15, 20],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 4],
#     "max_features": ["auto", "sqrt"]
# }

# rf_grid = GridSearchCV(
#     estimator=rf,
#     param_grid=rf_param_grid,
#     cv=5,
#     n_jobs=-1,
#     scoring="r2"
# )

# rf_grid.fit(X_train, Y_train)
# print(rf_grid.best_params_)

In [137]:
# import xgboost as xgb
# from sklearn.model_selection import GridSearchCV

# xgb_model = xgb.XGBRegressor(random_state=42)

# xgb_param_grid = {
#     "n_estimators": [300, 500, 700],
#     "max_depth": [3, 4, 5, 6],
#     "learning_rate": [0.01, 0.05, 0.1],
#     "subsample": [0.7, 0.8, 1.0],
#     "colsample_bytree": [0.7, 0.8, 1.0],
#     "reg_alpha": [0, 0.1, 0.5],
#     "reg_lambda": [1, 1.5, 2]
# }

# xgb_grid = GridSearchCV(
#     estimator=xgb_model,
#     param_grid=xgb_param_grid,
#     cv=5,
#     n_jobs=-1,
#     scoring="r2"
# )


# xgb_grid.fit(X_train, Y_train)
# print(xgb_grid.best_params_)

In [138]:
# from sklearn.linear_model import Ridge

# ridge = Ridge()

# ridge_param_grid = {
#     "alpha": [0.1, 1.0, 10, 50, 100]
# }

# ridge_grid = GridSearchCV(
#     estimator=ridge,
#     param_grid=ridge_param_grid,
#     cv=5,
#     n_jobs=-1,
#     scoring="r2"
# )

# ridge_grid.fit(X_train, Y_train)
# print(ridge_grid.best_params_)

In [139]:
# from sklearn.neural_network import MLPRegressor

# mlp = MLPRegressor(max_iter=1000, random_state=42, early_stopping=True)

# mlp_param_grid = {
#     "hidden_layer_sizes": [(32, 16), (64, 32), (64, 64, 32)],
#     "activation": ["relu", "tanh"],
#     "learning_rate_init": [0.001, 0.005, 0.01]
# }

# mlp_grid = GridSearchCV(
#     estimator=mlp,
#     param_grid=mlp_param_grid,
#     cv=5,
#     n_jobs=-1,
#     scoring="r2"
# )

# mlp_grid.fit(X_train, Y_train)
# print(mlp_grid.best_params_)

In [140]:
# from sklearn.model_selection import GridSearchCV

# svr = SVR(kernel='rbf')

# svr_param_grid = {
#    'C': [1, 10, 100],
#     'gamma': [0.01, 0.1, 1],
#     'epsilon': [0.01, 0.1, 0.2]
# }

# svr_grid = GridSearchCV(
#     estimator=svr,
#     param_grid=svr_param_grid,
#     cv=5,
#     n_jobs=-1,
#     scoring="r2"
# )

# svr_grid.fit(X_train, Y_train)
# print(svr_grid.best_params_)